<h3 align="center">NEU 437/537</h3>
<h4 align="center">Princeton University, Spring 2023</h4>

---
# Homework 4: Reinforcement Learning (39 points total)
#### Due: **Friday, May 5th at MIDNIGHT** (*10% off per day late*)
---

## Formatting Instructions
- Please prepare your homework submission completely within your own copy of this colab notebook.

- For each problem or sub-problem, please **limit yourself to one Code cell and/or one Markdown cell** as appropriate (switch between them by using the menu at the top, or the shortcuts `Ctrl+M M` for Markdown and `Ctrl+M B` for Code). 

- **Submitting your homework**:  Please submit an .ipynb file via the assignment tab in Canvas. (From your notebook, File->Download->Download .ipynb).  Late submissions will be penalized 10% per day.

- **Test before submmitting**: Before submitting, make sure to verify that your code runs without  errors by selecting `Runtime -> Restart & Run All`. 

- **Code Hygiene**: Giving variables human-readable names will make your code easier for both you and us to interpret. Similarly, when plotting, give your axes labels (using ```plt.xlabel()``` and ```plt.ylabel()```) and add legends where appropriate (using ```plt.legend()```).

## Set up
Let's import some of our favorite packages and set up some plotting parameters.

In [ ]:
%matplotlib inline
%pdb off

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.ndimage import gaussian_filter1d

# set default font sizes
SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 16
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

## Introduction


Reinforcement learning deals with the problem of how an intelligent agent can learn how to make decisions in an uncertain environment in order to maximize cumulative reward. Unlike in supervised learning, the learner is not told a priori what actions are rewarded. Instead, the agent must refine its action policies through trial and error. Reinforcement learning involves a fundamental trade-off between **exploration** of novel parts of the environment and **exploitation** of the agent's existing knowledge of the environment.

**In this problem set, you will explore 2 types of reinforcement learning problems: a multi-armed bandit task and a "Grid World" game**.

## Markov Decision Processes

*Note: This section gets a bit technical. Feel free to peruse it now and then refer back later when necessary.*

The environment in reinforcement learning is typically stated in the form of a **Markov decision process (MDP)**. This is a mathematical framework for descibing the **interaction between a decision-making agent and an environment that contains rewards**. 

<figure>
<img src="https://github.com/Brody-Lab/neu_437_public/raw/main/PSet4%20%20Reinforcement%20Learning/MDP.png" class="center" alt="Markov Decision Process" height=300></figure>

At each time step in an MDP, the process is in some state $S_t$, and the decision maker may choose any action $A_t$ that is available in state $S_t$. This action determines which state $S_{t+1}$ the process move to next, and gives the decision maker a corresponding reward $R_{t+1}$. The state transitions can be deterministic or stochastic. In this problem set, we will only consider deterministic state transitions -- that is, MDPs in which the agent's action completely determines the subsequent state.

In an MDP, state transitions are conditionally independent -- that is, the transition to the next state depends only on the current state and the agent's action -- thus satisfying the Markov property. In this way, MDPs extend the idea of Markov chains to include a decision-making agent and an environment that contains rewards.

Formally, a Markov decision process is a 4-tuple $(\mathcal{S},\mathcal{A},\mathcal{P_a},\mathcal{R_a})$, where:

- $\mathcal{S}$ is a set of states called the state space
- $\mathcal{A}$ is a set of actions called the action space
- $\mathcal{P_a}(s,s')$ is the probability of transitioning between two states s and s' given action $a$. These probabilities are either 1 or 0 for the problems we are considering today, since we are only considering fully deterministic state transitions.
- $\mathcal{R_a}(s,s')$ is the immediate reward (or expected immediate reward) received after transitioning from state $s$ to state $s'$ due to action $a$.

The goal of an agent in an MDP is to find an optimal action policy $\pi(s)$ that governs what action it takes in state $s$. The optimal policy $\pi^*$ is the one that maximizes its expected long-term return $G$:

$$\mathbb{E} \left[ G \right] = \mathbb{E} \left[ \sum_{t=0}^∞{\gamma^t R_{a_t}(s_t,s_{t+1}) } \right] = \mathbb{E} \left[ R_{a_0}(s_0,s_1) + \gamma R_{a_1}(s_1,s_2) + ... + \gamma^t R_{a_t}(s_t,s_{t+1})  \right] $$ where $a_t$ is the action given by the policy, i.e. $a_t = \pi(s_t)$.

Future rewards are discounted by some discounting factor $\gamma$ between $0$ and $1$. This discounting factor makes imminent rewards more important to the agent than distant rewards. There are a variety of motivations for the term that discounts future rewards. One is that distant rewards are more uncertain given our lack of complete knowledge of the environment.

## Question 1: The multi-armed bandit (20 points total)

In the reinforcement learning literature, the "multi-armed bandit" problem replicates the situation in which a gambler must repeatedly pull the lever of one of a series of slot machines ("one-armed bandits"), each of which is associated with some unknown (but stationary) reward probability distribution.

The agent must try to maximize their reward through trial and error, with each pull improving their estimate of the expected value of each bandit.

In this problem, you will simulate ten "bandits" and see how agents using a variety of reinforcement learning algorithms fare in maximizing reward.

<figure>
<img src="https://github.com/Brody-Lab/neu_437_public/raw/main/PSet4%20%20Reinforcement%20Learning/multi-armed bandit.png" class="center" alt="Markov Decision Process" height=300>,<figcaption>The "multi-armed bandit"</figcaption></figure>

### Question 1a: The multi-armed bandit as an MDP (2 points)

The multi-armed bandit problem be formalized as an MDP. Describe (in words) the set of states and actions in this MDP. Can you propose (in words) how an agent with no knowledge of the reward probabilities associated with the bandits could learn a good policy, simply through trial and error?

<font color="red">YOUR ANSWER IN RED TEXT HERE </font>

**(2 points)**

### Question 1b: Simulating 10 stationary "bandits" (3 points)

Let's say each bandit emits a reward drawn from a normal distribution with identical variance but a distinct mean. Therefore, the reward $R_t$ associated with pull $t$ is drawn from a normal distribution with mean $\mu_a$:   $$ R_t \sim \mathcal N(\mu_a, \sigma)$$

where $\mu_a$ is the mean reward associated with the action/bandit $a$ chosen on trial $t$ $A_t = a$. 

Create 10 such "bandits". To do this:

* First randomly generate their expected values by generating ten numbers at random from the interval [0,1] and storing them in the variable `mus`.

* Next define a function called `pull_arm` which takes a scalar input `mu` and returns a scalar output drawn from a normal distribution with mean `mu` and $\sigma$ = 0.1. This function simulates the reward dispensed by a bandit with expected value `mu`.

* Finally, calculate the bandit with highest expected value and store it as `best_bandit` and store its expected value as `best_mean_rate`.

Use the functions in [numpy.random](https://numpy.org/doc/stable/reference/random/index.html) to do the random number generation.

<font color="red"> YOUR ANSWER IN CODE BELOW </font> 

**(3 points)**

In [ ]:
###### YOUR CODE HERE #######

# Question 1b

n_bandits=10
mus = ...

def pull_arm(mu):
  ....

best_bandit = ...
best_mean_rate = ...

print("The best bandit is %d" % best_bandit)
print("Its mean reward rate is %0.2f" % best_mean_rate)

### Question 1c: The action-value function $Q$ (1 point)

Recall that, in our setup, the agent does not know the probability distributions governing the rewards dispensed by the bandits. Instead, the agent can only estimate their expected value by iteratively pulling one lever per trial $t$ and observing the reward $R_t$ it receives. 

Let's define the **action-value
function**
$$Q_t(a) = \mathbb{E}\left[R_{t} \mid A_{t}=a\right]$$
This defines the agent's estimate of the expected value of bandit $a$ on trial $t$, where $A_t$ is the action at time $t$ (i.e. pulling the arm of bandit $a$). This function depends on $t$ because this estimate is iteratively updated as the agent acquires information from the past sequence of rewards.

Describe in words how the agent might compute this estimate.

<font color="red"> YOUR ANSWER IN RED TEXT HERE </font>

**(1 point)**

### Question 1d: Bandit simulation using the greedy algorithm (6 points)
On each trial, there is always one action (i.e. one choice of bandit) that leads to the maximum estimated expected reward. This may not correspond to the bandit with the highest *actual* reward probability but is rather given by the action that maximizes the $Q$ function:

$$A_{t} = \underset{a}{\arg \max } Q_{t}(a)$$

This corresponds to the bandit the agent *currently thinks* will maximize expected reward. When the agent chooses this bandit, we say that it is using a **greedy** strategy. This strategy **exploits** its current knowledge of the
values of the actions.

In this question, you will simulate an agent using a greedy algorithm in the multi-armed bandit task. 

To do this:
* First, we'll define a function called `initialize_bandit_sim` which initializes some variables. Among other housekeeping tasks, this function initializes $Q$ by generating ten numbers at random from the interval [0,1]. This is just like how we initialized the means of the reward distributions of the ten bandits above, but now we are randomly initializing the agent's *estimate* of their expected values.
* Now we define a function called `one_bandit_sim` which simulates a variable number of sequential trials. On each trial, the agent first makes an action (i.e. chooses a bandit) according to the greedy algorithm described above. We keep track of the sequence of bandits chosen in a vector `A`. Then, using the function `pull_arm`, we pull the arm of the chosen bandit. This will generate the reward $R_t$ according to the distribution specified in Question 1a. We record the sequence of rewards received on each trial in a vector `R`. Finally, we update $Q_t(a)$ by calculating the average reward associated with the chosen action on trial $t$, i.e. $A_t = a$.
* In our main function `run_and_plot_bandit_sims` we repeat the above simulation several times. Each time, we plot the sequence of selected actions and the smoothed running average reward. For the final run, we also make a scatter plot of the actual versus final estimate of the expected value of each bandit. If the agent has found an optimal policy, they should have arrived at an accurate estimate of the expected value of each bandit.

The simulation code is largely already written, but needs to be completed.

<font color="red"> YOUR ANSWER IN CODE BELOW </font> 

**(3 points)**

In [ ]:
###### YOUR CODE HERE #######

# Question 1d

n_trials = 3000;

def initialize_bandit_sim():
  # initialize A and R
  A = np.empty(n_trials)
  R = np.empty(n_trials)
  A[:] = np.nan
  A=A.astype(int)
  Q =  ... # initialize Q, YOUR CODE HERE
  return A,R,Q

def one_bandit_sim():
  A,R,Q = initialize_bandit_sim();
  for t in range(n_trials): # simulate n_trials
    ### YOUR CODE HERE ####
    A[t] = ... # select the action for this trial using the greedy algorithm
    R[t] = ...  # pull the arm and record the received reward
    Q[A[t]] = ...  # update the Q function for the chosen action
  return A,R,Q

def run_and_plot_bandit_sims(n_reps=10,smoothing=25):
  fig, axs = plt.subplots(1, 3,figsize=(30,5))
  R_smooth = np.empty((n_trials,n_reps));
  for n in range(n_reps):
    A,R,Q = one_bandit_sim();
    plt.sca(axs[0])
    plt.plot(A+np.random.uniform(-0.1,0.1,size=n_trials),alpha=0.5) # plot sequence of chosen bandits (with jitter added for visibility)
    plt.sca(axs[1])
    R_smooth[:,n] = gaussian_filter1d(R,smoothing);
    plt.plot(R_smooth[:,n],alpha=0.5); # plot smoothed running average reward
  for i in range(2):
    plt.sca(axs[i])
    if (i==1) :
      plt.xlabel('Trial number')
      plt.ylabel('Average Reward')
    else:
      plt.ylabel('Chosen Bandit')
      plt.ylim(-0.1,0.1+n_bandits)   
    plt.xlabel('Trial')
    plt.xlim(0,n_trials)
    plt.grid()
  #
  plt.sca(axs[2])
  plt.plot(mus,Q,'.');
  plt.xlabel("bandit $\mu$")
  plt.ylabel("final Q (estimated expected value)")
  return np.mean(R_smooth,axis=1)

smooth_avg_reward_greedy = run_and_plot_bandit_sims();

Describe what you see in the plots above. Do different runs of the simulation lead to different sequences of choices and rates of reward? Does the greedy algorithm lead the agent to discover a good policy? Do the agent's Q values match the true reward values of the bandits? Can you think of different strategies an agent could use to improve their reward rate in the multi-arm bandit task? For your answers to all of these questions, explain **why** you gave the answer you did. 

<font color="red"> YOUR ANSWER IN RED TEXT HERE </font> 

**(3 points)**

### Question 1e: Bandit simulation using the $\epsilon$-greedy strategy (6 points)

An approximately optimal strategy in a multi-armed bandit task is an **$\epsilon$-greedy strategy**. This strategy selects the greedy action most of the time, but sometimes (i.e. with a small probability $\epsilon$) picks randomly. The value of epsilon defines the compromise between exploitation and exploration.

In this question, you will simulate an agent using a $\epsilon$-greedy algorithm.

To do this:
* Copy and paste the code defining the function `one_bandit_sim` from Question 1d. In the part of this function where the agent selects an action, add a conditional statement so that, at a probability given by `epsilon`, the action is selected randomly instead of using the greedy strategy.
* Rerun the main function `run_and_plot_bandit_sims` for $\epsilon=0.01,0.1,0.5$
* Then, in a single plot, plot the smoothed average reward rate for the greedy algorithm and the $\epsilon$-greedy algorithm for all three values of $\epsilon$. 
* Finally, in this plot, draw two horizontal dashed lines corresponding to the reward rate of the optimal policy (i.e. always choose the best bandit) and the reward rate of a fully random policy. Add a legend and label your axes.

<font color="red"> YOUR ANSWER IN CODE BELOW</font> 

**(3 points)**

In [ ]:
###### ANSWER #######

# Question 1e

# redefine the function "one_bandit_sim to use the epsilon-greedy strategy"
def one_bandit_sim():
  # YOUR CODE HERE

# rerun the simulation for epsilon = 0.01, 0.1, and 0.05
epsilon=0.01
smooth_avg_reward_epsilon_0_01 = run_and_plot_bandit_sims();  

epsilon=0.1
smooth_avg_reward_epsilon_0_1 = run_and_plot_bandit_sims();  

epsilon=0.5
smooth_avg_reward_epsilon_0_5 = run_and_plot_bandit_sims();  

# plot the smoothed average reward rate for the greedy algorithm and the epsilon-greedy algorithm for all three values of epsilon.
# draw two horizontal dashed lines corresponding to the reward rate of the optimal policy (i.e. always choose the best bandit) 
# and the reward rate of a fully random policy. Add a legend and label your axes.
# YOUR CODE HERE

How do the results of the $\epsilon$-greedy strategy compare to those of the greedy strategy (i.e. the $\epsilon$=0 strategy)? Which value of $\epsilon$ allows the agent to best estimate the reward values of the bandits? Which value of $\epsilon$ leads to the highest reward rate at the end of the simulation? Do any of the simulations achieve a final reward rate that approaches the optimal policy? Provide reasoning for why you observe these results. Can you think of a improvement to the epsilon-greedy algorithm that allows for more exploration in the early phase of the simulation to better learn the reward values of the bandits, and then more exploitation in the later phase once these reward values are learned?

<font color="red"> YOUR ANSWER IN RED TEXT HERE </font> 

**(3 points)**

### Question 1f: A real-life bandit problem (2 points) 

Discuss a situation in everyday life that can be accurately modeled using a multi-armed bandit problem. Discuss two types of decision-making problems that cannot be modeled as a multi-armed bandit problem.

<font color="red"> YOUR ANSWER IN RED TEXT HERE</font> 

**(2 points)**

## Question 2:  Grid World (19 points total)

The "Grid World" problem is a classic reinforcement learning problem in which an agent has to navigate through a discrete 2-D environment to search for reward. In the version we'll consider today, the agent starts in the top-left corner of a grid: $S_0 = (1,1)$. At each time step, the agent can take one of four actions: moving left, right, up or down. This deterministically transitions the agent to a new state/position in the environment, which can cause one of the four following outcomes:
* If the agent's action causes it to leave the grid, a negative reward (i.e. punishment) of -5 is delivered but the agent's position is unchanged.
* If the agent's action causes it to land on position (4,4) it receives a small reward of +10 and it is immediately transported back to the starting position.
* If it lands on (7,7) it receives a big reward of +100 and is transported back to the starting position.
* In all other cases, the agent receives a small negative reward of -1.

The small reward of +10 at position (4,4) is kind of like a "false goal". The agent is likely to discover this reward first, since it is closer to the start. But if the agent fails to further explore the environment, it may reach a local minimum of only pursuing this false goal instead of the much larger +100 reward at (7,7). In this problem, we'll play around with ways an agent could learn a policy in which it efficiently navigates to the true goal.

<figure>
<img src="https://github.com/Brody-Lab/neu_437_public/raw/main/PSet4%20%20Reinforcement%20Learning/grid world.PNG" class="center" height=500>,<figcaption>The setup of our "Grid World" problem</figcaption></figure>

### Question 2a: The "Grid World" problem as an MDP (3 points)

The "Grid World" problem can also be formalized as a Markov Decision Process. First, describe the state and action space of this MDP. How many states are there? How many actions? Next, describe the optimal policy in this MDP that our RL learning algorithm should learn. How many states does it take before the agent is transported to the start under this optimal policy? What is the average reward rate? What is the average reward rate for the policy that instead seeks the false goal as quickly as possible?

<font color="red"> YOUR ANSWER IN RED TEXT HERE</font> 

**(3 points)**

### Question 2b:  Simulating an agent using $\epsilon$-greedy in Grid World (5 points)

The "Grid World" problem involves a tradeoff between exploration and exploitation that is quite similar to what we saw in Question 1. An $\epsilon$-greedy strategy will therefore also be our starting point here. At each time step, our agent will choose an action that leads it to the position with the highest expected value, but will choose randomly with probability $\epsilon$. 

What's different here is that estimating state values is more complex. The position (6,7) delivers the same reward as position (1,1) but should be assigned much higher state-value since since a good policy will lead us to reward of +100 at the next time step.

This is why, in MDPs, we think of state-values as being more than simply an estimate of the reward available *in that state*. Instead, the state-value function in an MDP is the estimated long-term (discounted) return of being at a certain state $s$ while following a policy $\pi$
$$v^{\pi}(s) = \mathbb{E} \left[ G_t \mid S_{t}=s \right] = \mathbb{E} \left[ \sum_{i=0}^∞ \gamma^i R_{t+i} \mid S_{t}=s \right] $$

You will notice this formula is strikingly similar to the quantity that is maximized by the optimal policy $\pi^*$ (see the Introduction section). Indeed, the optimal policy is the one that *always* chooses the action that leads to the state with the maximum possible state-value. 

What's tricky is that the state-values of the optimal policy are hard to learn in practice since this requires having complete knowledge of the environment. Therefore, a major goal of reinforcement learning is to find an algorithm that approximately learns optimal state-values from trial-and-error feedback.

Temporal-difference (TD) learning provides a very simply, but often very effective, way to learn approximately-optimal state-values that we will take advantage of. Formally, TD learning is a way to estimate the value of a state $V(s)$ so that
$ V(s) \approx v^{\pi}(s)$

The simplest definition of the TD state-value estimate is: 
$$V\left(S_{t}\right) \leftarrow V\left(S_{t}\right)+\alpha \ \delta_t ,  \\  where \  \delta_t = R_{t}+    \gamma V(S_{t+1})-V(S_{t})  $$
$\delta_t$ is the error in our state-value estimate at time $t$. It is the difference between the current state-value estimate $V(S_{t})$ and the so-called "TD target", which is the sum of the actual reward we received $R_t$ and the discounted state-value estimate of the next state $\gamma V(S_{t+1})$. $\alpha$ is the learning rate (i.e. how much to update our value estimates at each time step given this error term). 

The code block below contains code that simulates the behavior of an agent in the Grid World problem defined above.

* First, we define some constants that specify the parameters of our Grid World environment. Then we define a function called `initialize_grid_world_sim` which initializes some variables. Among other housekeeping tasks, this function initializes the state-value estimates $V$ by generating a 7x7 grid of random numbers from the standard normal distribution. 
* Next we define a function called `state_transition` which defines how we move from one state to the next. It outputs the next state $S_{t+1}$ given our current state $S_t$, our state-value estimates $V$, and a value of $\epsilon$ that defines our probability of choosing an action randomly.
* Then we define a function `one_grid_world_sim` which simulates a variable number of sequential trials. On each trial, the agent first makes an action according to the epsilon-greedy algorithm by calling `state_transition`. The agent then receives a corresponding reward $R_t$ according to the rules outline above. Finally, we update our state-value estimates by calling `TD_learning` which implements the TD learning algorithm.
* In our main function `run_and_plot_grid_world_sims` we repeat the above simulation several times. Each time, we plot, as a 7x7 pixel image, the final state-value estimates $V$ and the smoothed running average reward for each of each of the `n_reps` runs. The function returns the grand average reward rate across the `n_reps` runs.

This time, the code is already completely written. Just run the cell below. You will then see how this agent, and modifications of this agent, behaves in the following sections.

In [ ]:
# ----- define grid world constants ------
grid_size = 7
actions = np.array([[1,0], [-1, 0], [0, 1], [0, -1]])  # possible actions (up, down, left and right)
n_actions = actions.shape[0]
start = np.array([0,0]) # define starting location
reward_avail = -np.ones((grid_size,grid_size)); # initialize available reward at all points to -1
reward_avail[3,3] = 10; # false goal reward
reward_avail[6,6] = 100; # true goal reward
leave_grid_punishment = -5 # punishment for leaving the grid

# ----- initialize the simulation ------
def initialize_grid_world_sim(n_timesteps):
  S = np.empty((n_timesteps+1,2),dtype=int);  # initialize the array to hold the sequence of states
  V = np.random.randn(grid_size,grid_size) # initialize value estimates for each state
  R = np.empty(n_timesteps) # initialize the array to hold the sequence of rewards obtained
  O = np.zeros((grid_size,grid_size)); # initialize array of state occupancy count
  S[0] = start; # state 0 is the starting position  
  return S,V,R,O

# ----- transition to next state -------
def state_transition(S,V,epsilon):
  if all(S==[6, 6]) or all(S==[3, 3]):
    # transport back to start
    S_next = start;
  else:
    # select actions according to the epsilon-greedy strategy  
    possible_S_next = S + actions;
    possible_S_next[possible_S_next>(grid_size-1)] = grid_size-1; # do not leave the grid
    possible_S_next[possible_S_next<0] = 0; # do not leave the grid
    if np.random.rand() < epsilon:
      # choose randomly with probability epsilon
      S_next = possible_S_next[np.random.choice(n_actions),:]; 
    else:
      # otherwise choose greedily
      possible_V_next = V[possible_S_next[:,0],possible_S_next[:,1]];
      S_next = possible_S_next[np.argmax(possible_V_next)];
  return S_next

# ----- update state-values using temporal difference learning -------
def TD_learning(S,V,R,t,alpha,gamma):
  error = (R[t] + gamma * V[tuple(S[t+1])] - V[tuple(S[t])]) ;
  V[tuple(S[t])] += alpha * error;
  return V

# ------ run one simulation ---------
def one_grid_world_sim(epsilon, gamma = 0.95, alpha=0.01, n_timesteps = int(3e4), smoothing=500):
  S,V,R,O = initialize_grid_world_sim(n_timesteps);
  for t in range(n_timesteps):   
    O[tuple(S[t])] += 1 # update state occupancy count
    S[t+1] = state_transition(S[t],V,epsilon);   
    # obtain reward
    if all(S[t+1] == S[t]) :
      # punish trying to leave the grid
      R[t] = leave_grid_punishment;
    else:
      # look up reward for this state
      R[t] = reward_avail[tuple(S[t])];
    # update value estimates using TD learning  
    V = TD_learning(S,V,R,t,alpha,gamma);  
  R_smooth = gaussian_filter1d(R,smoothing);
  S=S[:t+1]
  return S,V,R,R_smooth,O

# ------ run and plot several iterations of the simulation ------------------
# ------ with different starting points, return smoothed average reward -----
def run_and_plot_grid_world_sims(epsilon,n_reps=5,gamma=0.95,alpha=0.01,n_timesteps=int(3e4),smoothing=500):
  print("running grid world simulation %d times with epsilon = %0.1f" % (n_reps,epsilon))
  R_smooth = np.empty((n_timesteps,n_reps));
  fig, axs = plt.subplots(1, n_reps,figsize=(25,5));
  for i in range(n_reps):
    _,V,_,R_smooth[:,i],_ = one_grid_world_sim(epsilon,gamma,alpha,n_timesteps,smoothing);
    plt.sca(axs[i]);
    plt.imshow(V);
    plt.title("Final State-Value Estimates, run %d" % i);
  plt.figure(figsize=(15,4));
  plt.plot(R_smooth)    
  plt.title("epsilon=%0.1f"%epsilon)
  plt.xlabel("Timestep")
  plt.ylabel("Smoothed Reward Rate");  
  plt.ylim([-2,7])
  return np.mean(R_smooth,axis=1)

Run the code block above to define the constants and functions we'll need.

In the code block below, call the main function `run_and_plot_grid_world_sims` to run this simulation five times, first setting $\epsilon$ to $0$ to use the greedy strategy, and then using $0.1$ and finally $1$ to use completely random guessing. Then, in a single plot, plot the output (i.e. the smoothed average reward rate as a function of time) for each value of $\epsilon$.

<font color="red"> YOUR ANSWER IN CODE BELOW </font> 

**(2 points)**

In [ ]:
###### YOUR CODE HERE #######

# Question 2b

epsilons = [0, 0.1, 1];

# run the simulation for each value of epsilon
# YOUR CODE HERE

# plot smoothed average reward rate for each value of epsilon
# YOUR CODE HERE

Describe the results of running the above simulations. In particular, which value of $\epsilon$ gives the highest average reward rate? Does this rate of reward match any of the values you calculated in Question 2a? Which value of $\epsilon$ appears to give the most accurate reflection of the state-value of each position in Grid World? In the last question, a small value of $\epsilon$ allowed the agent to achieve a nearly optimal rate of reward? Is that the case here? Can you think of reasons why this is or isn't the case?

<font color="red"> YOUR ANSWER IN RED TEXT HERE</font> 

**(3 points)**

### Question 2c: Decaying $\epsilon$ (4 points)

One way to address the trade-off between exploration and exploitation contained in the kind of problems we are looking at is to use a **"decaying $\epsilon$ strategy"**. This involves starting out with a high value of $\epsilon$ to allow the agent to explore the environment. Then, as its knowledge of the environment improves, $\epsilon$ is decreased so that it can more effectively exploit that knowledge. A simple way to implement this is to let $\epsilon$ decay exponentially with a time constant $\tau$ ("tau"), such that $\epsilon_t = e^{-t/\tau} $. By this formulation, after every $\tau$ trials, the effective value of $\epsilon$ decays by a factor of $e$.

In the code block below, redefine the functions `one_grid_world_sim` and `run_and_plot_grid_world_sims` to implement a decaying $\epsilon$. To do this, replace all mentions of `epsilon` with `tau` except in the call to `state_transitions`. Here, you should instead replace `epsilon` with the definition of decaying-$\epsilon$ above.

Then run `run_and_plot_grid_world_sims` for several different values of $\tau$. 

*Note: since $\tau$ is the number of trials for epsilon to decay by a factor of $e$, it should be a relatively large number to give the agent a chance to explore. For the same reason set `n_timesteps=1e5` to lengthen the simulation.*

<font color="red"> YOUR ANSWER IN CODE BELOW

 </font> 

**(2 points)**

In [ ]:
###### YOUR CODE HERE #######

# Question 2c

# redefine "one_grid_world_smi" and "run_and_plot_grid_world_sims" to use tau instead of epsilon
# YOUR CODE HERE

# run grid world sim for various values of tau
# YOUR CODE HERE

Does the decaying-$\epsilon$ strategy help solve this Grid World problem? What maximum reward rate are you able to achieve? Are you able to approach the reward rate of the optimal policy? Explain what you think underlies these results.

<font color="red"> YOUR ANSWER IN RED TEXT HERE </font> 

**(2 points)**

### Question 2d: Multi-step TD learning (5 points)

You may have noticed that the agent in our Grid World simulations learns very slowly. One reason for this is the TD learning algorithm we've used so far is quite inefficient. Technically, what we've implemented is a particular version of TD learning called TD(0). TD(0) only propagates the state-value error by a single time step. TD(1) is a version of the algorithm that propagates error backward in time for all time steps as follows:

$$ V\left(S_{t-n}\right) \leftarrow V\left(S_{t-n}\right)+\alpha \gamma^{n-1} \delta_t\  ,    where \  \delta_t = R_{t}+    \gamma V(S_{t+1})-V(S_{t})  $$

If we apply this algorithm for only $n=0$, this algorithm reduces to TD(0). But we can also apply it in a loop for all $n$ up to some arbitrarily large number of steps back in time (even up to the beginning of the episode when $n=t$). To make it more concrete how TD(1) helps us more efficiently learn good state-values, consider what happens once the agent first experiences the +100 reward at the true goal. In TD(0), this information only affects the estimated value of the immediately-preceding state. For TD(1) we also update the estimated state-values of all the states that led us there, effectively enhancing the estimated value of the entire "path" to the reward.

In the code block below, redefine the function `TD_learning` to implement TD(1) instead of TD(0). Update state-values backward in time up to $n=50$. Then, run `run_and_plot_grid_world_sims` again for several different values of $\tau$. You will notice you can significantly shortern the simulation. Try `n_timesteps=15,000` and smaller values of `tau` than you used before.

<font color="red"> YOUR ANSWER IN CODE BELOW </font> 

**(3 points)**

In [ ]:
###### YOUR CODE HERE #######

# Question 2d

# redefine the function "TD_learning" to use TD(1) instead of TD(0). use maximum of n=50 steps back.
# YOUR CODE HERE

# rerun simulation for various (smaller) values of tau and n_timesteps=15,000
# YOUR CODE HERE

How long does it take the agent using decaying-$\epsilon$ to converge to a near-optimal solution now that you are using TD(1) to learn state-values? How does this compare to TD(0)? Give an explanation for why this is.

<font color="red"> YOUR ANSWER IN RED TEXT HERE </font> 

**(2 points)**

### Question 2e: Other solutions to Grid World (2 points)

If you were the agent doing this Grid World problem, how would you approach it? Would you use something like TD learning? What other algorithms could intelligent agents use to solve a problem like this? (Use your imagination -- there is no right answer!)

<font color="red"> YOUR ANSWER IN RED TEXT HERE </font> 

**(2 points)**